* 01.10.2022

* ver 0.3

* Yaromir Kapustin

### Задание
- Реализовать с использованием потоков и процессов скачивание файлов из интернета.
- Список файлов для скачивания подготовить самостоятельно (например изображений, не менее 100 изображений или других объектов.
- Сравнить производительность с последовательным методом. 
- Сравнивть производительность Thread и multiprocessing решений.
- Попробовать подобрать оптимальное число потоков/процессов. 


In [2]:
import time
import requests
from bs4 import BeautifulSoup
import concurrent.futures
from threading import Thread
from multiprocessing import Pool, cpu_count


images = []
def get_images_list(): #получаем список с ссылками на изображения
    list_url = ["https://unsplash.com/t/architecture"] #ресурс с множеством изображений
    for url in list_url:
        response = requests.get(url) #GET()-запрос на сайт и сохраним полученное в переменную 'response'; 200 или 404
        soup = BeautifulSoup(response.text, features="html.parser") #перевод ссылок в html формат
        for img in soup.findAll('img'): #поиск изображений
            img_src = img.get('src')
            if "images.unsplash.com/photo" in img_src: #фильтр лишних изображений
                images.append(img_src)
    return images

In [3]:
get_images_list()

['https://images.unsplash.com/photo-1664179532605-64fac359faef?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxwaG90by1vZi10aGUtZGF5fHx8fGVufDB8fHx8&auto=format%2Ccompress&fit=crop&w=1000&h=1000',
 'https://images.unsplash.com/photo-1664310748329-6710c43799a4?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHx0b3BpYy1mZWVkfDF8cm5TS0RId3dZVWt8fGVufDB8fHx8&w=1000&q=80',
 'https://images.unsplash.com/photo-1647103328346-c18478009ffa?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHx0b3BpYy1mZWVkfDR8cm5TS0RId3dZVWt8fGVufDB8fHx8&w=1000&q=80',
 'https://images.unsplash.com/photo-1664310748328-18b42c1c1725?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHx0b3BpYy1mZWVkfDZ8cm5TS0RId3dZVWt8fGVufDB8fHx8&w=1000&q=80',
 'https://images.unsplash.com/photo-1664384922834-c985a9659350?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHx0b3BpYy1mZWVkfDd8cm5TS0RId3dZVWt8fGVufDB8fHx8&w=1000&q=80',
 'https://images.unsplash.com/photo-1664189301709-52e9117be13e?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHx0b3BpYy1mZWVkfDl8cm5TS0RId3dZVWt8fGVufDB8fHx8&w=1000&q=80',
 'https://images.unsplash.

In [3]:
len(images) #кол-во ссылок

61

In [9]:
def sequential_method(): #применение последовательного метода
    start = time.time()
    for image in images:
        response = requests.get(image)
        print(image, response.status_code)
    end = time.time()
    print(end - start)

In [10]:
sequential_method()

https://images.unsplash.com/photo-1664179532605-64fac359faef?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxwaG90by1vZi10aGUtZGF5fHx8fGVufDB8fHx8&auto=format%2Ccompress&fit=crop&w=1000&h=1000 200
https://images.unsplash.com/photo-1664310748329-6710c43799a4?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHx0b3BpYy1mZWVkfDF8cm5TS0RId3dZVWt8fGVufDB8fHx8&w=1000&q=80 200
https://images.unsplash.com/photo-1647103328346-c18478009ffa?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHx0b3BpYy1mZWVkfDR8cm5TS0RId3dZVWt8fGVufDB8fHx8&w=1000&q=80 200
https://images.unsplash.com/photo-1664310748328-18b42c1c1725?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHx0b3BpYy1mZWVkfDZ8cm5TS0RId3dZVWt8fGVufDB8fHx8&w=1000&q=80 200
https://images.unsplash.com/photo-1664384922834-c985a9659350?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHx0b3BpYy1mZWVkfDd8cm5TS0RId3dZVWt8fGVufDB8fHx8&w=1000&q=80 200
https://images.unsplash.com/photo-1664189301709-52e9117be13e?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHx0b3BpYy1mZWVkfDl8cm5TS0RId3dZVWt8fGVufDB8fHx8&w=1000&q=80 200
https://images.unsplash.co

> Выполнение последовательного метода заняло 279 сек

In [4]:
def get_status(image): #функция получения статуса изображения для передачи в метод
    response = requests.get(image, verify=False) #Отключение верификации ускорило выполнение алгоритма!
    return image

In [4]:
def threadpoolexecutor_method():
    start = time.time()
    with concurrent.futures.ThreadPoolExecutor(max_workers=16) as executor:
        future_to_url = {executor.submit(get_status, image): image for image in images}
        for future in concurrent.futures.as_completed(future_to_url):
            url = future_to_url[future]
            print(future.result())
    end = time.time()
    print(end - start)

In [5]:
threadpoolexecutor_method()

C:\conda\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'images.unsplash.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\conda\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'images.unsplash.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\conda\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'images.unsplash.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\conda\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified 

https://images.unsplash.com/photo-1664310748328-18b42c1c1725?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHx0b3BpYy1mZWVkfDZ8cm5TS0RId3dZVWt8fGVufDB8fHx8&w=1000&q=80
https://images.unsplash.com/photo-1664179532605-64fac359faef?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxwaG90by1vZi10aGUtZGF5fHx8fGVufDB8fHx8&auto=format%2Ccompress&fit=crop&w=1000&h=1000
https://images.unsplash.com/photo-1664384922834-c985a9659350?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHx0b3BpYy1mZWVkfDd8cm5TS0RId3dZVWt8fGVufDB8fHx8&w=1000&q=80


C:\conda\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'images.unsplash.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\conda\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'images.unsplash.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


https://images.unsplash.com/photo-1664310748329-6710c43799a4?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHx0b3BpYy1mZWVkfDF8cm5TS0RId3dZVWt8fGVufDB8fHx8&w=1000&q=80


C:\conda\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'images.unsplash.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


...

https://images.unsplash.com/photo-1656911052309-4e1ccc0146f9?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHx0b3BpYy1mZWVkfDN8cm5TS0RId3dZVWt8fGVufDB8fHx8&w=1000&q=80
https://images.unsplash.com/photo-1664032304991-c029bb4d161d?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHx0b3BpYy1mZWVkfDEyfHJuU0tESHd3WVVrfHxlbnwwfHx8fA%3D%3D&w=1000&q=80
https://images.unsplash.com/photo-1664332184517-9dc41ea0085a?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHx0b3BpYy1mZWVkfDh8cm5TS0RId3dZVWt8fGVufDB8fHx8&w=1000&q=80


C:\conda\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'images.unsplash.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


https://images.unsplash.com/photo-1664032365338-c7b6e393dda1?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHx0b3BpYy1mZWVkfDE0fHJuU0tESHd3WVVrfHxlbnwwfHx8fA%3D%3D&w=1000&q=80
https://images.unsplash.com/photo-1656911051433-a2c0fb2985c6?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHx0b3BpYy1mZWVkfDE3fHJuU0tESHd3WVVrfHxlbnwwfHx8fA%3D%3D&w=1000&q=80


C:\conda\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'images.unsplash.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\conda\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'images.unsplash.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


https://images.unsplash.com/photo-1664332184517-9dc41ea0085a?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHx0b3BpYy1mZWVkfDh8cm5TS0RId3dZVWt8fGVufDB8fHx8&w=1000&q=80
https://images.unsplash.com/photo-1653378972356-223320488772?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHx0b3BpYy1mZWVkfDE2fHJuU0tESHd3WVVrfHxlbnwwfHx8fA%3D%3D&w=1000&q=80
https://images.unsplash.com/photo-1664179532605-64fac359faef?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHx0b3BpYy1mZWVkfDEwfHJuU0tESHd3WVVrfHxlbnwwfHx8fA%3D%3D&w=1000&q=80
https://images.unsplash.com/photo-1664384922834-c985a9659350?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHx0b3BpYy1mZWVkfDd8cm5TS0RId3dZVWt8fGVufDB8fHx8&w=1000&q=80
https://images.unsplash.com/photo-1656911051485-b49ad9513135?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHx0b3BpYy1mZWVkfDE4fHJuU0tESHd3WVVrfHxlbnwwfHx8fA%3D%3D&w=1000&q=80
https://images.unsplash.com/photo-1664095682837-aba6a1706e7c?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHx0b3BpYy1mZWVkfDExfHJuU0tESHd3WVVrfHxlbnwwfHx8fA%3D%3D&w=1000&q=80
https://images.unsplash.com/photo-1662541351

> Выполнение многопоточного метода заняло: 

* 55 сек при 4 потоках, 
* 36 сек при 8 потоках, 
* 32 сек при 16 потоках

In [ ]:
def pool_handler(): #многопоточный метод
    start = time.time()
    p = Pool(1)
    print(p.map(get_status, images))
    end = time.time()
    print(end - start)


if __name__ == '__main__':
    pool_handler()

In [ ]:
multiprocessing_method()

> Выполнение многопроцессорного метода заняло: 

* 32 сек при использовании 1 ядра, 
* 22 сек при использовании 2 ядер, 
* 29 сек при использовании 2 ядер, 
* 23 сек при использовании 8 ядер, 
* 24 сек при использовании 16 ядер

-- Метод не выполняется через JN, но можно через PyCharm

## Выводы

В моём случае оказался наиболее предпочтительным метод мультипроцессинга при использовании 2 ядер. Многопоточный метод также может быть использован (быстрее последовательного в 10 раз). Отключение верификации сертификата HTTPS в случае многопоточного метода может ускорить алгоритма.